In [1]:
from playsound import playsound
from scipy.io import wavfile
import numpy as np
import os
import matplotlib.pyplot as plt
import IPython
import cv2
import pandas as pd

# Data Collection/Exploration

In [2]:
#gets us to the folder with the recordings
os.chdir(r'C:\Users\Alex\Desktop\Programming\DSP\Python\Audio\machine learning\free-spoken-digit-dataset-master\recordings')
wavList = os.listdir()
print(len(wavList))

#get labels for all files
labels = []
for file in wavList:
    labels.append(file[0])

3000


In [ ]:
inspect = 501
samplerate, data = wavfile.read(wavList[inspect])
plt.plot(data)
plt.show()
plt.specgram(data)
plt.show()
IPython.display.Audio(wavList[inspect])

In [ ]:
for inspect in range(100):
    samplerate, data = wavfile.read(wavList[inspect])
    plt.plot(data)
    plt.show()
    plt.specgram(data)
    plt.show()

In [ ]:
#approach: classify sounds using the same techniques to classify images, i.e. we use the spectrogram as our image!
#obtain our spectrograms
currNum = 0
currVoice = 0
path = r'C:\Users\Alex\Desktop\Programming\DSP\Python\Audio\machine learning\free-spoken-digit-dataset-master\spectrograms'
for file in wavList:
    samplerate, data = wavfile.read(file)
    
    if currVoice != file[0]:
        currVoice = file[0]
        currNum = 0
    
    name = '\\'+str(currVoice)+str(currNum).zfill(3)
    plt.specgram(data)
    
    #uncomment below to save everything
    print(path+name)
    #plt.savefig(path+name)
    
    currNum += 1
    plt.clf()

# Machine Learning Data Preparation

In [3]:
#gets us to the folder with the images
path = r'C:\Users\Alex\Desktop\Programming\DSP\Python\Audio\machine learning\free-spoken-digit-dataset-master\spectrograms'
pathCSV = r'C:\Users\Alex\Desktop\Programming\DSP\Python\Audio\machine learning\free-spoken-digit-dataset-master\csv\data.csv'
os.chdir(path)
imgList = os.listdir()
print(len(imgList))
print(labels[0:10])

#inspect saved image
img = cv2.imread(imgList[0],cv2.IMREAD_GRAYSCALE)
print(img.shape)

#collect all images
images = []
for img in imgList:
    images.append(img)
    
#create pandas dataset with label and image
data = {'filename':images,'label':labels}
df = pd.DataFrame(data)

3000
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
(288, 432)


In [4]:
df.head()

,filename,label
0,0000.png,0
1,0001.png,0
2,0002.png,0
3,0003.png,0
4,0004.png,0


In [43]:
#uncomment to save a csv file
df.to_csv(pathCSV, index = False)

# Pytorch Use

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data

In [45]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [46]:
#hyperparameters
in_channel = 3
num_classes = 10
learning_rate = 1e-3
batch_size = 32
num_epochs = 4

In [47]:
#since this is inheriting from the dataset class, it should have the right setup for pytorch!
#https://stackoverflow.com/questions/65231299/load-csv-and-image-dataset-in-pytorch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, transform = None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform
        self.class2index = {"0":0, "1":1, "2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9}

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        filename = self.df[index, "FILENAME"]
        label = self.class2index[self.df[index, "LABEL"]]
        image = PIL.Image.open(os.path.join(self.images_folder, filename))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [48]:
imgPath = r'C:\Users\Alex\Desktop\Programming\DSP\Python\Audio\machine learning\free-spoken-digit-dataset-master\spectrograms'
csvPath = r'C:\Users\Alex\Desktop\Programming\DSP\Python\Audio\machine learning\free-spoken-digit-dataset-master\csv\data.csv'

#pytorch data preparation
dataset = CustomDataset(csvPath, imgPath)

In [49]:
#pytorch data preparation
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*.8), len(dataset)-int(len(dataset)*.8)])
train_loader = data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

train_set[222]

KeyError: (341, 'FILENAME')

In [ ]:
#using a pretrained model
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [27]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

KeyError: (1552, 'FILENAME')